In [4]:
'''
temporal out of sample validation
'''

from hydroml.config.config import Config
from hydroml.workflow.evaluation import train_finetune_evaluate
import multiprocessing
import hydroml.utils.helpers as h

from hydroml.utils import config_path_handler as cp

PLATFORM = 'win'


def get_toos_cross_validation_dates(fold, all_folds, start_date='1970-01-01', end_date='2015-01-01', buffer_years=5):

    val_period = all_folds[str(fold)]
    cal_period = [[start_date, val_period[0]],
                   [h.add_years_to_date(val_period[1], buffer_years), end_date]]
    
    return cal_period, [val_period]
    

    
def get_config(name, fold):
    all_folds = h.read_json(cp.get_package_base_dir() / 'examples' / 'sample_data' / '4fold.json')
    basins_file = cp.get_package_base_dir() / 'examples' / 'sample_data' / 'basins.txt'

    with open(basins_file, 'r') as f:
        catchment_ids = f.read().splitlines()

    
    catchment_ids = catchment_ids[:2]
    cal_period, val_period= get_toos_cross_validation_dates(fold, all_folds)
    config = Config(cal={'periods' : cal_period, 'catchment_ids':catchment_ids}, 
                    val={'periods' : val_period, 'catchment_ids':catchment_ids}, 
                    name = f'{name}_toos_xval_{fold}_of_{len(all_folds)}',
                    device='cpu',
                    platform='win'
                    )    
    
    if name == 'toos_qc_validation':
        config.dynamic_features = ['precipitation_AWAP', 'et_morton_wet_SILO', 'qtot']
    
    elif name == 'toos_c_validation':
        config.dynamic_features = ['precipitation_AWAP', 'et_morton_wet_SILO']
    
    elif name == 'toos_qc_validation_silo_silo':
        config.dynamic_features = ['precipitation_SILO', 'et_morton_wet_SILO', 'qtot']
    
    elif name == 'toos_c_validation_silo_silo':
        config.dynamic_features = ['precipitation_SILO', 'et_morton_wet_SILO']
    
    elif name == 'toos_q_validation':
        config.dynamic_features = ['qtot']    


    config.set_new_version_name()

    return config


def main(name, fold):
    print(f'xval: running {name} for fold {fold}')
    
    config = get_config(name, fold)
    train_finetune_evaluate(config)





In [8]:
from hydroml.data import get_dataset

config = get_config('toos_qc_validation', 0 )
dataset = get_dataset(config, 'val')
dataset

Transforming data: calculating transform parameters and saving to P:\work\sho108\hydroml\results\toos_qc_validation_toos_xval_0_of_4\241216161628_6788\params.yaml


<xarray.Dataset> Size: 1MB
Dimensions:          (catchment_id: 2, date: 3654, dynamic_feature: 25,
                      static_feature_dim: 13, metadata_dim: 1)
Coordinates:
  * date             (date) datetime64[ns] 29kB 1975-01-01 ... 1985-01-01
  * dynamic_feature  (dynamic_feature) <U22 2kB 'precipitation_AWAP' ... 'qtot'
  * metadata_dim     (metadata_dim) <U6 24B 'weight'
  * catchment_id     (catchment_id) <U7 56B '912101A' '912105A'
Dimensions without coordinates: static_feature_dim
Data variables:
    dynamic          (catchment_id, date, dynamic_feature) float64 1MB 14.77 ...
    static           (catchment_id, static_feature_dim) float64 208B 1.258e+0...
    metadata         (catchment_id, metadata_dim) int32 8B 1 1
Attributes:
    weight:   1

In [ ]:
dataset

In [2]:
main('toos_qc_validation')

{'parent_path': 'P://work//sho108//hydroml//results', 'camels_aus_attributes': 'Z://Data//CAMELS_AUS//CAMELS_AUS_Attributes&Indices_MasterTable.csv', 'camels_aus_v1': 'Z://Data//CAMELS_AUS//preprocessed', 'awra_qtot_postprocessed': 'L://work//sho108//AWRA//historical//v1//AWRALv7//preprocessed_catchment_mean//qtot_3'}


Transforming data: calculating transform parameters and saving to P:\work\sho108\hydroml\results\toos_qc_validation\241216104834_e4f4\params.yaml


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\torch\utils\data\dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


valid data points per catchment {0: 5381, 1: 2219}


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\torch\utils\data\dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results\toos_qc_validation\241216104834_e4f4\params.yaml
P:\work\sho108\hydroml\results\toos_qc_validation\241216104834_e4f4


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory \\fs1-cbr.nexus.csiro.au\{ev-ca-macq}\work\sho108\hydroml\results\toos_qc_validation\241216104834_e4f4 exists and is not empty.

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | static_embedding  | Linear     | 15     | train
1 | dynamic_embedding | Linear     | 6      | train
2 | lstm              | LSTM       | 266 K  | train
3 | dropout           | Identity   | 0      | train
4 | head              | Sequential | 2.6 K  | train
---------------------------------------------------------
268 K     Trainable params
0         Non-trainable params
268 K     Total params
1.075     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined